In [26]:
import os
import math
import sqlite3

import numpy as np
import pandas as pd

db_path = os.path.join(os.path.dirname("__file__"), "..", "..", "data", "ufc.db")

In [27]:
query = """
WITH cte1 AS (
    SELECT
        t1.fighter_id,
        t1.'order',
        t1.event_id,
        t1.opponent_id,
        ROW_NUMBER() OVER (PARTITION BY t1.fighter_id, t1.event_id, t1.opponent_id ORDER BY t1.'order') AS temp_rn,
        CASE
            WHEN t1.outcome = 'W' THEN 1
            ELSE 0
        END AS win,
        CASE
            WHEN t1.outcome = 'L' THEN 1
            ELSE 0
        END AS lose,
        CASE
            WHEN t2.country LIKE '% USA%' THEN 'united states'
            ELSE LOWER(t2.country)
        END AS country,
        CASE
            WHEN t3.nationality = 'USA' THEN 'united states'
            ELSE LOWER(t3.nationality)
        END AS nationality,
        CASE
            WHEN t4.nationality = 'USA' THEN 'united states'
            ELSE LOWER(t4.nationality)
        END AS opp_nationality
    FROM
        sherdog_fighter_histories t1
    LEFT JOIN sherdog_events t2 ON t1.event_id = t2.id
    LEFT JOIN sherdog_fighters t3 ON t1.fighter_id = t3.id
    LEFT JOIN sherdog_fighters t4 ON t1.opponent_id = t4.id
),
cte2 AS (
    SELECT
        event_id,
        bout_order,
        fighter_1_id AS fighter_id,
        fighter_2_id AS opponent_id,
        is_title_bout,
        weight_class
    FROM
        sherdog_bouts
    UNION
    SELECT
        event_id,
        bout_order,
        fighter_2_id AS fighter_id,
        fighter_1_id AS opponent_id,
        is_title_bout,
        weight_class
    FROM
        sherdog_bouts
),
cte3 AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY event_id, fighter_id, opponent_id ORDER BY bout_order) AS temp_rn
    FROM
        cte2
),
cte4 AS (
    SELECT
        t1.fighter_id,
        t1.'order',
        t1.event_id,
        t1.opponent_id,
        t1.temp_rn,
        t1.win,
        t1.lose,
        t1.country,
        t1.opp_nationality,
        CASE
            WHEN t1.nationality = t1.country THEN 1
            ELSE 0
        END AS is_home_country,
        CASE
            WHEN t1.opp_nationality = t1.country THEN 1
            ELSE 0
        END AS opp_is_home_country,
        t2.is_title_bout,
        t2.weight_class
    FROM
        cte1 t1
    LEFT JOIN cte3 t2 ON t1.event_id = t2.event_id AND t1.fighter_id = t2.fighter_id AND t1.opponent_id = t2.opponent_id AND t1.temp_rn = t2.temp_rn
    ORDER BY t1.fighter_id, t1.'order'
),
cte5 AS (
    SELECT
        fighter_id,
        t1.'order',
        event_id,
        opponent_id,
        t1.temp_rn,
        SUM(win) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins,
        SUM(lose) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses,
        AVG(win) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct,
        AVG(lose) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct,
        SUM(win) OVER (
            PARTITION BY fighter_id, country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_by_country,
        SUM(lose) OVER (
            PARTITION BY fighter_id, country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_by_country,
        AVG(win) OVER (
            PARTITION BY fighter_id, country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_by_country,
        AVG(lose) OVER (
            PARTITION BY fighter_id, country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_by_country,
        SUM(win) OVER (
            PARTITION BY fighter_id, opp_nationality
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_against_opp_nationality,
        SUM(lose) OVER (
            PARTITION BY fighter_id, opp_nationality
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_against_opp_nationality,
        AVG(win) OVER (
            PARTITION BY fighter_id, opp_nationality
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_against_opp_nationality,
        AVG(lose) OVER (
            PARTITION BY fighter_id, opp_nationality
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_against_opp_nationality,
        SUM(win) OVER (
            PARTITION BY fighter_id, is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_by_is_home_country,
        SUM(lose) OVER (
            PARTITION BY fighter_id, is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_by_is_home_country,
        AVG(win) OVER (
            PARTITION BY fighter_id, is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_by_is_home_country,
        AVG(lose) OVER (
            PARTITION BY fighter_id, is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_by_is_home_country,
        SUM(win) OVER (
            PARTITION BY fighter_id, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_by_opp_is_home_country,
        SUM(lose) OVER (
            PARTITION BY fighter_id, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_by_opp_is_home_country,
        AVG(win) OVER (
            PARTITION BY fighter_id, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_by_opp_is_home_country,
        AVG(lose) OVER (
            PARTITION BY fighter_id, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_by_opp_is_home_country,
        SUM(win) OVER (
            PARTITION BY fighter_id, is_home_country, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_by_is_home_country_and_opp_is_home_country,
        SUM(lose) OVER (
            PARTITION BY fighter_id, is_home_country, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_by_is_home_country_and_opp_is_home_country,
        AVG(win) OVER (
            PARTITION BY fighter_id, is_home_country, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_by_is_home_country_and_opp_is_home_country,
        AVG(lose) OVER (
            PARTITION BY fighter_id, is_home_country, opp_is_home_country
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_by_is_home_country_and_opp_is_home_country,
        SUM(win) OVER (
            PARTITION BY fighter_id, is_title_bout
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_by_is_title_bout,
        SUM(lose) OVER (
            PARTITION BY fighter_id, is_title_bout
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_by_is_title_bout,
        AVG(win) OVER (
            PARTITION BY fighter_id, is_title_bout
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_by_is_title_bout,
        AVG(lose) OVER (
            PARTITION BY fighter_id, is_title_bout
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_by_is_title_bout,
        SUM(win) OVER (
            PARTITION BY fighter_id, weight_class
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS wins_by_weight_class,
        SUM(lose) OVER (
            PARTITION BY fighter_id, weight_class
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS losses_by_weight_class,
        AVG(win) OVER (
            PARTITION BY fighter_id, weight_class
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS win_pct_by_weight_class,
        AVG(lose) OVER (
            PARTITION BY fighter_id, weight_class
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS lose_pct_by_weight_class
    FROM
        cte4 t1
    ORDER BY t1.fighter_id, t1.'order'
),
cte6 AS (
    SELECT
        fighter_id,
        t1.'order',
        event_id,
        opponent_id,
        t1.temp_rn,
        CASE
            WHEN wins IS NULL THEN 0
            ELSE wins
        END AS wins,
        CASE
            WHEN losses IS NULL THEN 0
            ELSE losses
        END AS losses,
        win_pct,
        lose_pct,
        CASE
            WHEN wins_by_country IS NULL THEN 0
            ELSE wins_by_country
        END AS wins_by_country,
        CASE
            WHEN losses_by_country IS NULL THEN 0
            ELSE losses_by_country
        END AS losses_by_country,
        CASE
            WHEN win_pct_by_country IS NULL THEN win_pct
            ELSE win_pct_by_country
        END AS win_pct_by_country,
        CASE
            WHEN lose_pct_by_country IS NULL THEN lose_pct
            ELSE lose_pct_by_country
        END AS lose_pct_by_country,
        CASE
            WHEN wins_against_opp_nationality IS NULL THEN 0
            ELSE wins_against_opp_nationality
        END AS wins_against_opp_nationality,
        CASE
            WHEN losses_against_opp_nationality IS NULL THEN 0
            ELSE losses_against_opp_nationality
        END AS losses_against_opp_nationality,
        CASE
            WHEN win_pct_against_opp_nationality IS NULL THEN win_pct
            ELSE win_pct_against_opp_nationality
        END AS win_pct_against_opp_nationality,
        CASE
            WHEN lose_pct_against_opp_nationality IS NULL THEN lose_pct
            ELSE lose_pct_against_opp_nationality
        END AS lose_pct_against_opp_nationality,
        CASE
            WHEN wins_by_is_home_country IS NULL THEN 0
            ELSE wins_by_is_home_country
        END AS wins_by_is_home_country,
        CASE
            WHEN losses_by_is_home_country IS NULL THEN 0
            ELSE losses_by_is_home_country
        END AS losses_by_is_home_country,
        CASE
            WHEN win_pct_by_is_home_country IS NULL THEN win_pct
            ELSE win_pct_by_is_home_country
        END AS win_pct_by_is_home_country,
        CASE
            WHEN lose_pct_by_is_home_country IS NULL THEN lose_pct
            ELSE lose_pct_by_is_home_country
        END AS lose_pct_by_is_home_country,
        CASE
            WHEN wins_by_opp_is_home_country IS NULL THEN 0
            ELSE wins_by_opp_is_home_country
        END AS wins_by_opp_is_home_country,
        CASE
            WHEN losses_by_opp_is_home_country IS NULL THEN 0
            ELSE losses_by_opp_is_home_country
        END AS losses_by_opp_is_home_country,
        CASE
            WHEN win_pct_by_opp_is_home_country IS NULL THEN win_pct
            ELSE win_pct_by_opp_is_home_country
        END AS win_pct_by_opp_is_home_country,
        CASE
            WHEN lose_pct_by_opp_is_home_country IS NULL THEN lose_pct
            ELSE lose_pct_by_opp_is_home_country
        END AS lose_pct_by_opp_is_home_country,
        CASE
            WHEN wins_by_is_home_country_and_opp_is_home_country IS NULL THEN 0
            ELSE wins_by_is_home_country_and_opp_is_home_country
        END AS wins_by_is_home_country_and_opp_is_home_country,
        CASE
            WHEN losses_by_is_home_country_and_opp_is_home_country IS NULL THEN 0
            ELSE losses_by_is_home_country_and_opp_is_home_country
        END AS losses_by_is_home_country_and_opp_is_home_country,
        CASE
            WHEN win_pct_by_is_home_country_and_opp_is_home_country IS NULL THEN win_pct
            ELSE win_pct_by_is_home_country_and_opp_is_home_country
        END AS win_pct_by_is_home_country_and_opp_is_home_country,
        CASE
            WHEN lose_pct_by_is_home_country_and_opp_is_home_country IS NULL THEN lose_pct
            ELSE lose_pct_by_is_home_country_and_opp_is_home_country
        END AS lose_pct_by_is_home_country_and_opp_is_home_country,
        CASE
            WHEN wins_by_is_title_bout IS NULL THEN 0
            ELSE wins_by_is_title_bout
        END AS wins_by_is_title_bout,
        CASE
            WHEN losses_by_is_title_bout IS NULL THEN 0
            ELSE losses_by_is_title_bout
        END AS losses_by_is_title_bout,
        CASE
            WHEN win_pct_by_is_title_bout IS NULL THEN win_pct
            ELSE win_pct_by_is_title_bout
        END AS win_pct_by_is_title_bout,
        CASE
            WHEN lose_pct_by_is_title_bout IS NULL THEN lose_pct
            ELSE lose_pct_by_is_title_bout
        END AS lose_pct_by_is_title_bout,
        CASE
            WHEN wins_by_weight_class IS NULL THEN 0
            ELSE wins_by_weight_class
        END AS wins_by_weight_class,
        CASE
            WHEN losses_by_weight_class IS NULL THEN 0
            ELSE losses_by_weight_class
        END AS losses_by_weight_class,
        CASE
            WHEN win_pct_by_weight_class IS NULL THEN win_pct
            ELSE win_pct_by_weight_class
        END AS win_pct_by_weight_class,
        CASE
            WHEN lose_pct_by_weight_class IS NULL THEN lose_pct
            ELSE lose_pct_by_weight_class
        END AS lose_pct_by_weight_class
    FROM
        cte5 t1
),
cte7 AS (
    SELECT
        t1.fighter_id,
        t1.'order',
        t1.event_id,
        t1.opponent_id,
        t1.wins_by_country,
        t1.losses_by_country,
        t1.win_pct_by_country,
        t1.lose_pct_by_country,
        t1.wins_against_opp_nationality,
        t1.losses_against_opp_nationality,
        t1.win_pct_against_opp_nationality,
        t1.lose_pct_against_opp_nationality,
        t1.wins_by_is_home_country,
        t1.losses_by_is_home_country,
        t1.win_pct_by_is_home_country,
        t1.lose_pct_by_is_home_country,
        t1.wins_by_opp_is_home_country,
        t1.losses_by_opp_is_home_country,
        t1.win_pct_by_opp_is_home_country,
        t1.lose_pct_by_opp_is_home_country,
        t1.wins_by_is_home_country_and_opp_is_home_country,
        t1.losses_by_is_home_country_and_opp_is_home_country,
        t1.win_pct_by_is_home_country_and_opp_is_home_country,
        t1.lose_pct_by_is_home_country_and_opp_is_home_country,
        t1.wins_by_is_title_bout,
        t1.losses_by_is_title_bout,
        t1.win_pct_by_is_title_bout,
        t1.lose_pct_by_is_title_bout,
        t1.wins_by_weight_class,
        t1.losses_by_weight_class,
        t1.win_pct_by_weight_class,
        t1.lose_pct_by_weight_class,
        AVG(t2.wins_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_country,
        AVG(t1.wins_by_country - t2.wins_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_country_diff,
        AVG(1.0 * t1.wins_by_country / t2.wins_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_country_ratio,
        AVG(t2.losses_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_country,
        AVG(t1.losses_by_country - t2.losses_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_country_diff,
        AVG(1.0 * t1.losses_by_country / t2.losses_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_country_ratio,
        AVG(t2.win_pct_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_country,
        AVG(t1.win_pct_by_country - t2.win_pct_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_country_diff,
        AVG(1.0 * t1.win_pct_by_country / t2.win_pct_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_country_ratio,
        AVG(t2.lose_pct_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_country,
        AVG(t1.lose_pct_by_country - t2.lose_pct_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_country_diff,
        AVG(1.0 * t1.lose_pct_by_country / t2.lose_pct_by_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_country_ratio,
        AVG(t2.wins_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_against_opp_nationality,
        AVG(t1.wins_against_opp_nationality - t2.wins_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_against_opp_nationality_diff,
        AVG(1.0 * t1.wins_against_opp_nationality / t2.wins_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_against_opp_nationality_ratio,
        AVG(t2.losses_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_against_opp_nationality,
        AVG(t1.losses_against_opp_nationality - t2.losses_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_against_opp_nationality_diff,
        AVG(1.0 * t1.losses_against_opp_nationality / t2.losses_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_against_opp_nationality_ratio,
        AVG(t2.win_pct_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_against_opp_nationality,
        AVG(t1.win_pct_against_opp_nationality - t2.win_pct_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_against_opp_nationality_diff,
        AVG(1.0 * t1.win_pct_against_opp_nationality / t2.win_pct_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_against_opp_nationality_ratio,
        AVG(t2.lose_pct_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_against_opp_nationality,
        AVG(t1.lose_pct_against_opp_nationality - t2.lose_pct_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_against_opp_nationality_diff,
        AVG(1.0 * t1.lose_pct_against_opp_nationality / t2.lose_pct_against_opp_nationality) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_against_opp_nationality_ratio,
        AVG(t2.wins_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_home_country,
        AVG(t1.wins_by_is_home_country - t2.wins_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_home_country_diff,
        AVG(1.0 * t1.wins_by_is_home_country / t2.wins_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_home_country_ratio,
        AVG(t2.losses_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_home_country,
        AVG(t1.losses_by_is_home_country - t2.losses_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_home_country_diff,
        AVG(1.0 * t1.losses_by_is_home_country / t2.losses_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_home_country_ratio,
        AVG(t2.win_pct_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_home_country,
        AVG(t1.win_pct_by_is_home_country - t2.win_pct_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_home_country_diff,
        AVG(1.0 * t1.win_pct_by_is_home_country / t2.win_pct_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_home_country_ratio,
        AVG(t2.lose_pct_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_home_country,
        AVG(t1.lose_pct_by_is_home_country - t2.lose_pct_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_home_country_diff,
        AVG(1.0 * t1.lose_pct_by_is_home_country / t2.lose_pct_by_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_home_country_ratio,
        AVG(t2.wins_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_opp_is_home_country,
        AVG(t1.wins_by_opp_is_home_country - t2.wins_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_opp_is_home_country_diff,
        AVG(1.0 * t1.wins_by_opp_is_home_country / t2.wins_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_opp_is_home_country_ratio,
        AVG(t2.losses_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_opp_is_home_country,
        AVG(t1.losses_by_opp_is_home_country - t2.losses_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_opp_is_home_country_diff,
        AVG(1.0 * t1.losses_by_opp_is_home_country / t2.losses_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_opp_is_home_country_ratio,
        AVG(t2.win_pct_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_opp_is_home_country,
        AVG(t1.win_pct_by_opp_is_home_country - t2.win_pct_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_opp_is_home_country_diff,
        AVG(1.0 * t1.win_pct_by_opp_is_home_country / t2.win_pct_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_opp_is_home_country_ratio,
        AVG(t2.lose_pct_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_opp_is_home_country,
        AVG(t1.lose_pct_by_opp_is_home_country - t2.lose_pct_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_opp_is_home_country_diff,
        AVG(1.0 * t1.lose_pct_by_opp_is_home_country / t2.lose_pct_by_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_opp_is_home_country_ratio,
        AVG(t2.wins_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_home_country_and_opp_is_home_country,
        AVG(t1.wins_by_is_home_country_and_opp_is_home_country - t2.wins_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff,
        AVG(1.0 * t1.wins_by_is_home_country_and_opp_is_home_country / t2.wins_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio,
        AVG(t2.losses_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_home_country_and_opp_is_home_country,
        AVG(t1.losses_by_is_home_country_and_opp_is_home_country - t2.losses_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff,
        AVG(1.0 * t1.losses_by_is_home_country_and_opp_is_home_country / t2.losses_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio,
        AVG(t2.win_pct_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country,
        AVG(t1.win_pct_by_is_home_country_and_opp_is_home_country - t2.win_pct_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff,
        AVG(1.0 * t1.win_pct_by_is_home_country_and_opp_is_home_country / t2.win_pct_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio,
        AVG(t2.lose_pct_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country,
        AVG(t1.lose_pct_by_is_home_country_and_opp_is_home_country - t2.lose_pct_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff,
        AVG(1.0 * t1.lose_pct_by_is_home_country_and_opp_is_home_country / t2.lose_pct_by_is_home_country_and_opp_is_home_country) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio,
        AVG(t2.wins_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_title_bout,
        AVG(t1.wins_by_is_title_bout - t2.wins_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_title_bout_diff,
        AVG(1.0 * t1.wins_by_is_title_bout / t2.wins_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_is_title_bout_ratio,
        AVG(t2.losses_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_title_bout,
        AVG(t1.losses_by_is_title_bout - t2.losses_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_title_bout_diff,
        AVG(1.0 * t1.losses_by_is_title_bout / t2.losses_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_is_title_bout_ratio,
        AVG(t2.win_pct_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_title_bout,
        AVG(t1.win_pct_by_is_title_bout - t2.win_pct_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_title_bout_diff,
        AVG(1.0 * t1.win_pct_by_is_title_bout / t2.win_pct_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_is_title_bout_ratio,
        AVG(t2.lose_pct_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_title_bout,
        AVG(t1.lose_pct_by_is_title_bout - t2.lose_pct_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_title_bout_diff,
        AVG(1.0 * t1.lose_pct_by_is_title_bout / t2.lose_pct_by_is_title_bout) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_is_title_bout_ratio,
        AVG(t2.wins_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_weight_class,
        AVG(t1.wins_by_weight_class - t2.wins_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_weight_class_diff,
        AVG(1.0 * t1.wins_by_weight_class / t2.wins_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_wins_by_weight_class_ratio,
        AVG(t2.losses_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_weight_class,
        AVG(t1.losses_by_weight_class - t2.losses_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_weight_class_diff,
        AVG(1.0 * t1.losses_by_weight_class / t2.losses_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_losses_by_weight_class_ratio,
        AVG(t2.win_pct_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_weight_class,
        AVG(t1.win_pct_by_weight_class - t2.win_pct_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_weight_class_diff,
        AVG(1.0 * t1.win_pct_by_weight_class / t2.win_pct_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_win_pct_by_weight_class_ratio,
        AVG(t2.lose_pct_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_weight_class,
        AVG(t1.lose_pct_by_weight_class - t2.lose_pct_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_weight_class_diff,
        AVG(1.0 * t1.lose_pct_by_weight_class / t2.lose_pct_by_weight_class) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_lose_pct_by_weight_class_ratio
    FROM
        cte6 t1
    LEFT JOIN cte6 t2 ON t1.fighter_id = t2.opponent_id AND t1.event_id = t2.event_id AND t1.opponent_id = t2.fighter_id AND t1.temp_rn = t2.temp_rn
),
cte8 AS (
    SELECT
        t2.ufcstats_id AS fighter_id,
        t1.'order',
        t4.ufcstats_id AS event_id,
        t3.ufcstats_id AS opponent_id,
        t1.wins_by_country,
        t1.losses_by_country,
        t1.win_pct_by_country,
        t1.lose_pct_by_country,
        t1.wins_against_opp_nationality,
        t1.losses_against_opp_nationality,
        t1.win_pct_against_opp_nationality,
        t1.lose_pct_against_opp_nationality,
        t1.wins_by_is_home_country,
        t1.losses_by_is_home_country,
        t1.win_pct_by_is_home_country,
        t1.lose_pct_by_is_home_country,
        t1.wins_by_opp_is_home_country,
        t1.losses_by_opp_is_home_country,
        t1.win_pct_by_opp_is_home_country,
        t1.lose_pct_by_opp_is_home_country,
        t1.wins_by_is_home_country_and_opp_is_home_country,
        t1.losses_by_is_home_country_and_opp_is_home_country,
        t1.win_pct_by_is_home_country_and_opp_is_home_country,
        t1.lose_pct_by_is_home_country_and_opp_is_home_country,
        t1.wins_by_is_title_bout,
        t1.losses_by_is_title_bout,
        t1.win_pct_by_is_title_bout,
        t1.lose_pct_by_is_title_bout,
        t1.wins_by_weight_class,
        t1.losses_by_weight_class,
        t1.win_pct_by_weight_class,
        t1.lose_pct_by_weight_class,
        t1.avg_opp_wins_by_country,
        t1.avg_opp_wins_by_country_diff,
        t1.avg_opp_wins_by_country_ratio,
        t1.avg_opp_losses_by_country,
        t1.avg_opp_losses_by_country_diff,
        t1.avg_opp_losses_by_country_ratio,
        t1.avg_opp_win_pct_by_country,
        t1.avg_opp_win_pct_by_country_diff,
        t1.avg_opp_win_pct_by_country_ratio,
        t1.avg_opp_lose_pct_by_country,
        t1.avg_opp_lose_pct_by_country_diff,
        t1.avg_opp_lose_pct_by_country_ratio,
        t1.avg_opp_wins_against_opp_nationality,
        t1.avg_opp_wins_against_opp_nationality_diff,
        t1.avg_opp_wins_against_opp_nationality_ratio,
        t1.avg_opp_losses_against_opp_nationality,
        t1.avg_opp_losses_against_opp_nationality_diff,
        t1.avg_opp_losses_against_opp_nationality_ratio,
        t1.avg_opp_win_pct_against_opp_nationality,
        t1.avg_opp_win_pct_against_opp_nationality_diff,
        t1.avg_opp_win_pct_against_opp_nationality_ratio,
        t1.avg_opp_lose_pct_against_opp_nationality,
        t1.avg_opp_lose_pct_against_opp_nationality_diff,
        t1.avg_opp_lose_pct_against_opp_nationality_ratio,
        t1.avg_opp_wins_by_is_home_country,
        t1.avg_opp_wins_by_is_home_country_diff,
        t1.avg_opp_wins_by_is_home_country_ratio,
        t1.avg_opp_losses_by_is_home_country,
        t1.avg_opp_losses_by_is_home_country_diff,
        t1.avg_opp_losses_by_is_home_country_ratio,
        t1.avg_opp_win_pct_by_is_home_country,
        t1.avg_opp_win_pct_by_is_home_country_diff,
        t1.avg_opp_win_pct_by_is_home_country_ratio,
        t1.avg_opp_lose_pct_by_is_home_country,
        t1.avg_opp_lose_pct_by_is_home_country_diff,
        t1.avg_opp_lose_pct_by_is_home_country_ratio,
        t1.avg_opp_wins_by_opp_is_home_country,
        t1.avg_opp_wins_by_opp_is_home_country_diff,
        t1.avg_opp_wins_by_opp_is_home_country_ratio,
        t1.avg_opp_losses_by_opp_is_home_country,
        t1.avg_opp_losses_by_opp_is_home_country_diff,
        t1.avg_opp_losses_by_opp_is_home_country_ratio,
        t1.avg_opp_win_pct_by_opp_is_home_country,
        t1.avg_opp_win_pct_by_opp_is_home_country_diff,
        t1.avg_opp_win_pct_by_opp_is_home_country_ratio,
        t1.avg_opp_lose_pct_by_opp_is_home_country,
        t1.avg_opp_lose_pct_by_opp_is_home_country_diff,
        t1.avg_opp_lose_pct_by_opp_is_home_country_ratio,
        t1.avg_opp_wins_by_is_home_country_and_opp_is_home_country,
        t1.avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff,
        t1.avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio,
        t1.avg_opp_losses_by_is_home_country_and_opp_is_home_country,
        t1.avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff,
        t1.avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio,
        t1.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country,
        t1.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff,
        t1.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio,
        t1.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country,
        t1.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff,
        t1.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio,
        t1.avg_opp_wins_by_is_title_bout,
        t1.avg_opp_wins_by_is_title_bout_diff,
        t1.avg_opp_wins_by_is_title_bout_ratio,
        t1.avg_opp_losses_by_is_title_bout,
        t1.avg_opp_losses_by_is_title_bout_diff,
        t1.avg_opp_losses_by_is_title_bout_ratio,
        t1.avg_opp_win_pct_by_is_title_bout,
        t1.avg_opp_win_pct_by_is_title_bout_diff,
        t1.avg_opp_win_pct_by_is_title_bout_ratio,
        t1.avg_opp_lose_pct_by_is_title_bout,
        t1.avg_opp_lose_pct_by_is_title_bout_diff,
        t1.avg_opp_lose_pct_by_is_title_bout_ratio,
        t1.avg_opp_wins_by_weight_class,
        t1.avg_opp_wins_by_weight_class_diff,
        t1.avg_opp_wins_by_weight_class_ratio,
        t1.avg_opp_losses_by_weight_class,
        t1.avg_opp_losses_by_weight_class_diff,
        t1.avg_opp_losses_by_weight_class_ratio,
        t1.avg_opp_win_pct_by_weight_class,
        t1.avg_opp_win_pct_by_weight_class_diff,
        t1.avg_opp_win_pct_by_weight_class_ratio,
        t1.avg_opp_lose_pct_by_weight_class,
        t1.avg_opp_lose_pct_by_weight_class_diff,
        t1.avg_opp_lose_pct_by_weight_class_ratio
    FROM
        cte7 t1
    INNER JOIN
        fighter_mapping AS t2 ON t1.fighter_id = t2.sherdog_id
    INNER JOIN
        fighter_mapping AS t3 ON t1.opponent_id = t3.sherdog_id
    INNER JOIN
        event_mapping AS t4 ON t1.event_id = t4.sherdog_id
),
cte9 AS (
    SELECT
        fighter_id,
        ROW_NUMBER() OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
        ) AS ufc_order,
        event_id,
        opponent_id,
        wins_by_country,
        losses_by_country,
        win_pct_by_country,
        lose_pct_by_country,
        wins_against_opp_nationality,
        losses_against_opp_nationality,
        win_pct_against_opp_nationality,
        lose_pct_against_opp_nationality,
        wins_by_is_home_country,
        losses_by_is_home_country,
        win_pct_by_is_home_country,
        lose_pct_by_is_home_country,
        wins_by_opp_is_home_country,
        losses_by_opp_is_home_country,
        win_pct_by_opp_is_home_country,
        lose_pct_by_opp_is_home_country,
        wins_by_is_home_country_and_opp_is_home_country,
        losses_by_is_home_country_and_opp_is_home_country,
        win_pct_by_is_home_country_and_opp_is_home_country,
        lose_pct_by_is_home_country_and_opp_is_home_country,
        wins_by_is_title_bout,
        losses_by_is_title_bout,
        win_pct_by_is_title_bout,
        lose_pct_by_is_title_bout,
        wins_by_weight_class,
        losses_by_weight_class,
        win_pct_by_weight_class,
        lose_pct_by_weight_class,
        avg_opp_wins_by_country,
        avg_opp_wins_by_country_diff,
        avg_opp_wins_by_country_ratio,
        avg_opp_losses_by_country,
        avg_opp_losses_by_country_diff,
        avg_opp_losses_by_country_ratio,
        avg_opp_win_pct_by_country,
        avg_opp_win_pct_by_country_diff,
        avg_opp_win_pct_by_country_ratio,
        avg_opp_lose_pct_by_country,
        avg_opp_lose_pct_by_country_diff,
        avg_opp_lose_pct_by_country_ratio,
        avg_opp_wins_against_opp_nationality,
        avg_opp_wins_against_opp_nationality_diff,
        avg_opp_wins_against_opp_nationality_ratio,
        avg_opp_losses_against_opp_nationality,
        avg_opp_losses_against_opp_nationality_diff,
        avg_opp_losses_against_opp_nationality_ratio,
        avg_opp_win_pct_against_opp_nationality,
        avg_opp_win_pct_against_opp_nationality_diff,
        avg_opp_win_pct_against_opp_nationality_ratio,
        avg_opp_lose_pct_against_opp_nationality,
        avg_opp_lose_pct_against_opp_nationality_diff,
        avg_opp_lose_pct_against_opp_nationality_ratio,
        avg_opp_wins_by_is_home_country,
        avg_opp_wins_by_is_home_country_diff,
        avg_opp_wins_by_is_home_country_ratio,
        avg_opp_losses_by_is_home_country,
        avg_opp_losses_by_is_home_country_diff,
        avg_opp_losses_by_is_home_country_ratio,
        avg_opp_win_pct_by_is_home_country,
        avg_opp_win_pct_by_is_home_country_diff,
        avg_opp_win_pct_by_is_home_country_ratio,
        avg_opp_lose_pct_by_is_home_country,
        avg_opp_lose_pct_by_is_home_country_diff,
        avg_opp_lose_pct_by_is_home_country_ratio,
        avg_opp_wins_by_opp_is_home_country,
        avg_opp_wins_by_opp_is_home_country_diff,
        avg_opp_wins_by_opp_is_home_country_ratio,
        avg_opp_losses_by_opp_is_home_country,
        avg_opp_losses_by_opp_is_home_country_diff,
        avg_opp_losses_by_opp_is_home_country_ratio,
        avg_opp_win_pct_by_opp_is_home_country,
        avg_opp_win_pct_by_opp_is_home_country_diff,
        avg_opp_win_pct_by_opp_is_home_country_ratio,
        avg_opp_lose_pct_by_opp_is_home_country,
        avg_opp_lose_pct_by_opp_is_home_country_diff,
        avg_opp_lose_pct_by_opp_is_home_country_ratio,
        avg_opp_wins_by_is_home_country_and_opp_is_home_country,
        avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff,
        avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio,
        avg_opp_losses_by_is_home_country_and_opp_is_home_country,
        avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff,
        avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio,
        avg_opp_win_pct_by_is_home_country_and_opp_is_home_country,
        avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff,
        avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio,
        avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country,
        avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff,
        avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio,
        avg_opp_wins_by_is_title_bout,
        avg_opp_wins_by_is_title_bout_diff,
        avg_opp_wins_by_is_title_bout_ratio,
        avg_opp_losses_by_is_title_bout,
        avg_opp_losses_by_is_title_bout_diff,
        avg_opp_losses_by_is_title_bout_ratio,
        avg_opp_win_pct_by_is_title_bout,
        avg_opp_win_pct_by_is_title_bout_diff,
        avg_opp_win_pct_by_is_title_bout_ratio,
        avg_opp_lose_pct_by_is_title_bout,
        avg_opp_lose_pct_by_is_title_bout_diff,
        avg_opp_lose_pct_by_is_title_bout_ratio,
        avg_opp_wins_by_weight_class,
        avg_opp_wins_by_weight_class_diff,
        avg_opp_wins_by_weight_class_ratio,
        avg_opp_losses_by_weight_class,
        avg_opp_losses_by_weight_class_diff,
        avg_opp_losses_by_weight_class_ratio,
        avg_opp_win_pct_by_weight_class,
        avg_opp_win_pct_by_weight_class_diff,
        avg_opp_win_pct_by_weight_class_ratio,
        avg_opp_lose_pct_by_weight_class,
        avg_opp_lose_pct_by_weight_class_diff,
        avg_opp_lose_pct_by_weight_class_ratio
    FROM
        cte8 t1
),
cte10 AS (
    SELECT
        t1.*
    FROM
        ufcstats_fighter_histories AS t1
    LEFT JOIN
        ufcstats_bouts AS t2
    ON
        t1.bout_id = t2.id
    LEFT JOIN
        ufcstats_events AS t3
    ON
        t2.event_id = t3.id
    WHERE
        t3.is_ufc_event = 1
),
cte11 AS (
        SELECT
        fighter_id,
        ROW_NUMBER() OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
        ) AS ufc_order,
        bout_id,
        opponent_id
    FROM
        cte10 t1
),
cte12 AS (
    SELECT
        t1.fighter_id,
        t1.bout_id,
        t2.wins_by_country,
        t2.losses_by_country,
        t2.win_pct_by_country,
        t2.lose_pct_by_country,
        t2.wins_against_opp_nationality,
        t2.losses_against_opp_nationality,
        t2.win_pct_against_opp_nationality,
        t2.lose_pct_against_opp_nationality,
        t2.wins_by_is_home_country,
        t2.losses_by_is_home_country,
        t2.win_pct_by_is_home_country,
        t2.lose_pct_by_is_home_country,
        t2.wins_by_opp_is_home_country,
        t2.losses_by_opp_is_home_country,
        t2.win_pct_by_opp_is_home_country,
        t2.lose_pct_by_opp_is_home_country,
        t2.wins_by_is_home_country_and_opp_is_home_country,
        t2.losses_by_is_home_country_and_opp_is_home_country,
        t2.win_pct_by_is_home_country_and_opp_is_home_country,
        t2.lose_pct_by_is_home_country_and_opp_is_home_country,
        t2.wins_by_is_title_bout,
        t2.losses_by_is_title_bout,
        t2.win_pct_by_is_title_bout,
        t2.lose_pct_by_is_title_bout,
        t2.wins_by_weight_class,
        t2.losses_by_weight_class,
        t2.win_pct_by_weight_class,
        t2.lose_pct_by_weight_class,
        t2.avg_opp_wins_by_country,
        t2.avg_opp_wins_by_country_diff,
        t2.avg_opp_wins_by_country_ratio,
        t2.avg_opp_losses_by_country,
        t2.avg_opp_losses_by_country_diff,
        t2.avg_opp_losses_by_country_ratio,
        t2.avg_opp_win_pct_by_country,
        t2.avg_opp_win_pct_by_country_diff,
        t2.avg_opp_win_pct_by_country_ratio,
        t2.avg_opp_lose_pct_by_country,
        t2.avg_opp_lose_pct_by_country_diff,
        t2.avg_opp_lose_pct_by_country_ratio,
        t2.avg_opp_wins_against_opp_nationality,
        t2.avg_opp_wins_against_opp_nationality_diff,
        t2.avg_opp_wins_against_opp_nationality_ratio,
        t2.avg_opp_losses_against_opp_nationality,
        t2.avg_opp_losses_against_opp_nationality_diff,
        t2.avg_opp_losses_against_opp_nationality_ratio,
        t2.avg_opp_win_pct_against_opp_nationality,
        t2.avg_opp_win_pct_against_opp_nationality_diff,
        t2.avg_opp_win_pct_against_opp_nationality_ratio,
        t2.avg_opp_lose_pct_against_opp_nationality,
        t2.avg_opp_lose_pct_against_opp_nationality_diff,
        t2.avg_opp_lose_pct_against_opp_nationality_ratio,
        t2.avg_opp_wins_by_is_home_country,
        t2.avg_opp_wins_by_is_home_country_diff,
        t2.avg_opp_wins_by_is_home_country_ratio,
        t2.avg_opp_losses_by_is_home_country,
        t2.avg_opp_losses_by_is_home_country_diff,
        t2.avg_opp_losses_by_is_home_country_ratio,
        t2.avg_opp_win_pct_by_is_home_country,
        t2.avg_opp_win_pct_by_is_home_country_diff,
        t2.avg_opp_win_pct_by_is_home_country_ratio,
        t2.avg_opp_lose_pct_by_is_home_country,
        t2.avg_opp_lose_pct_by_is_home_country_diff,
        t2.avg_opp_lose_pct_by_is_home_country_ratio,
        t2.avg_opp_wins_by_opp_is_home_country,
        t2.avg_opp_wins_by_opp_is_home_country_diff,
        t2.avg_opp_wins_by_opp_is_home_country_ratio,
        t2.avg_opp_losses_by_opp_is_home_country,
        t2.avg_opp_losses_by_opp_is_home_country_diff,
        t2.avg_opp_losses_by_opp_is_home_country_ratio,
        t2.avg_opp_win_pct_by_opp_is_home_country,
        t2.avg_opp_win_pct_by_opp_is_home_country_diff,
        t2.avg_opp_win_pct_by_opp_is_home_country_ratio,
        t2.avg_opp_lose_pct_by_opp_is_home_country,
        t2.avg_opp_lose_pct_by_opp_is_home_country_diff,
        t2.avg_opp_lose_pct_by_opp_is_home_country_ratio,
        t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country,
        t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff,
        t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio,
        t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country,
        t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff,
        t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio,
        t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country,
        t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff,
        t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio,
        t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country,
        t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff,
        t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio,
        t2.avg_opp_wins_by_is_title_bout,
        t2.avg_opp_wins_by_is_title_bout_diff,
        t2.avg_opp_wins_by_is_title_bout_ratio,
        t2.avg_opp_losses_by_is_title_bout,
        t2.avg_opp_losses_by_is_title_bout_diff,
        t2.avg_opp_losses_by_is_title_bout_ratio,
        t2.avg_opp_win_pct_by_is_title_bout,
        t2.avg_opp_win_pct_by_is_title_bout_diff,
        t2.avg_opp_win_pct_by_is_title_bout_ratio,
        t2.avg_opp_lose_pct_by_is_title_bout,
        t2.avg_opp_lose_pct_by_is_title_bout_diff,
        t2.avg_opp_lose_pct_by_is_title_bout_ratio,
        t2.avg_opp_wins_by_weight_class,
        t2.avg_opp_wins_by_weight_class_diff,
        t2.avg_opp_wins_by_weight_class_ratio,
        t2.avg_opp_losses_by_weight_class,
        t2.avg_opp_losses_by_weight_class_diff,
        t2.avg_opp_losses_by_weight_class_ratio,
        t2.avg_opp_win_pct_by_weight_class,
        t2.avg_opp_win_pct_by_weight_class_diff,
        t2.avg_opp_win_pct_by_weight_class_ratio,
        t2.avg_opp_lose_pct_by_weight_class,
        t2.avg_opp_lose_pct_by_weight_class_diff,
        t2.avg_opp_lose_pct_by_weight_class_ratio
    FROM
        cte11 t1
    INNER JOIN
        cte9 t2 ON t1.fighter_id = t2.fighter_id AND t1.opponent_id = t2.opponent_id AND t1.ufc_order = t2.ufc_order
)
SELECT
    id,
    t2.wins_by_country - t3.wins_by_country AS wins_by_country_diff,
    1.0 * t2.wins_by_country / t3.wins_by_country AS wins_by_country_ratio,
    t2.losses_by_country - t3.losses_by_country AS losses_by_country_diff,
    1.0 * t2.losses_by_country / t3.losses_by_country AS losses_by_country_ratio,
    t2.win_pct_by_country - t3.win_pct_by_country AS win_pct_by_country_diff,
    1.0 * t2.win_pct_by_country / t3.win_pct_by_country AS win_pct_by_country_ratio,
    t2.lose_pct_by_country - t3.lose_pct_by_country AS lose_pct_by_country_diff,
    1.0 * t2.lose_pct_by_country / t3.lose_pct_by_country AS lose_pct_by_country_ratio,
    t2.wins_against_opp_nationality - t3.wins_against_opp_nationality AS wins_against_opp_nationality_diff,
    1.0 * t2.wins_against_opp_nationality / t3.wins_against_opp_nationality AS wins_against_opp_nationality_ratio,
    t2.losses_against_opp_nationality - t3.losses_against_opp_nationality AS losses_against_opp_nationality_diff,
    1.0 * t2.losses_against_opp_nationality / t3.losses_against_opp_nationality AS losses_against_opp_nationality_ratio,
    t2.win_pct_against_opp_nationality - t3.win_pct_against_opp_nationality AS win_pct_against_opp_nationality_diff,
    1.0 * t2.win_pct_against_opp_nationality / t3.win_pct_against_opp_nationality AS win_pct_against_opp_nationality_ratio,
    t2.lose_pct_against_opp_nationality - t3.lose_pct_against_opp_nationality AS lose_pct_against_opp_nationality_diff,
    1.0 * t2.lose_pct_against_opp_nationality / t3.lose_pct_against_opp_nationality AS lose_pct_against_opp_nationality_ratio,
    t2.wins_by_is_home_country - t3.wins_by_is_home_country AS wins_by_is_home_country_diff,
    1.0 * t2.wins_by_is_home_country / t3.wins_by_is_home_country AS wins_by_is_home_country_ratio,
    t2.losses_by_is_home_country - t3.losses_by_is_home_country AS losses_by_is_home_country_diff,
    1.0 * t2.losses_by_is_home_country / t3.losses_by_is_home_country AS losses_by_is_home_country_ratio,
    t2.win_pct_by_is_home_country - t3.win_pct_by_is_home_country AS win_pct_by_is_home_country_diff,
    1.0 * t2.win_pct_by_is_home_country / t3.win_pct_by_is_home_country AS win_pct_by_is_home_country_ratio,
    t2.lose_pct_by_is_home_country - t3.lose_pct_by_is_home_country AS lose_pct_by_is_home_country_diff,
    1.0 * t2.lose_pct_by_is_home_country / t3.lose_pct_by_is_home_country AS lose_pct_by_is_home_country_ratio,
    t2.wins_by_opp_is_home_country - t3.wins_by_opp_is_home_country AS wins_by_opp_is_home_country_diff,
    1.0 * t2.wins_by_opp_is_home_country / t3.wins_by_opp_is_home_country AS wins_by_opp_is_home_country_ratio,
    t2.losses_by_opp_is_home_country - t3.losses_by_opp_is_home_country AS losses_by_opp_is_home_country_diff,
    1.0 * t2.losses_by_opp_is_home_country / t3.losses_by_opp_is_home_country AS losses_by_opp_is_home_country_ratio,
    t2.win_pct_by_opp_is_home_country - t3.win_pct_by_opp_is_home_country AS win_pct_by_opp_is_home_country_diff,
    1.0 * t2.win_pct_by_opp_is_home_country / t3.win_pct_by_opp_is_home_country AS win_pct_by_opp_is_home_country_ratio,
    t2.lose_pct_by_opp_is_home_country - t3.lose_pct_by_opp_is_home_country AS lose_pct_by_opp_is_home_country_diff,
    1.0 * t2.lose_pct_by_opp_is_home_country / t3.lose_pct_by_opp_is_home_country AS lose_pct_by_opp_is_home_country_ratio,
    t2.wins_by_is_home_country_and_opp_is_home_country - t3.wins_by_is_home_country_and_opp_is_home_country AS wins_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.wins_by_is_home_country_and_opp_is_home_country / t3.wins_by_is_home_country_and_opp_is_home_country AS wins_by_is_home_country_and_opp_is_home_country_ratio,
    t2.losses_by_is_home_country_and_opp_is_home_country - t3.losses_by_is_home_country_and_opp_is_home_country AS losses_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.losses_by_is_home_country_and_opp_is_home_country / t3.losses_by_is_home_country_and_opp_is_home_country AS losses_by_is_home_country_and_opp_is_home_country_ratio,
    t2.win_pct_by_is_home_country_and_opp_is_home_country - t3.win_pct_by_is_home_country_and_opp_is_home_country AS win_pct_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.win_pct_by_is_home_country_and_opp_is_home_country / t3.win_pct_by_is_home_country_and_opp_is_home_country AS win_pct_by_is_home_country_and_opp_is_home_country_ratio,
    t2.lose_pct_by_is_home_country_and_opp_is_home_country - t3.lose_pct_by_is_home_country_and_opp_is_home_country AS lose_pct_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.lose_pct_by_is_home_country_and_opp_is_home_country / t3.lose_pct_by_is_home_country_and_opp_is_home_country AS lose_pct_by_is_home_country_and_opp_is_home_country_ratio,
    t2.wins_by_is_title_bout - t3.wins_by_is_title_bout AS wins_by_is_title_bout_diff,
    1.0 * t2.wins_by_is_title_bout / t3.wins_by_is_title_bout AS wins_by_is_title_bout_ratio,
    t2.losses_by_is_title_bout - t3.losses_by_is_title_bout AS losses_by_is_title_bout_diff,
    1.0 * t2.losses_by_is_title_bout / t3.losses_by_is_title_bout AS losses_by_is_title_bout_ratio,
    t2.win_pct_by_is_title_bout - t3.win_pct_by_is_title_bout AS win_pct_by_is_title_bout_diff,
    1.0 * t2.win_pct_by_is_title_bout / t3.win_pct_by_is_title_bout AS win_pct_by_is_title_bout_ratio,
    t2.lose_pct_by_is_title_bout - t3.lose_pct_by_is_title_bout AS lose_pct_by_is_title_bout_diff,
    1.0 * t2.lose_pct_by_is_title_bout / t3.lose_pct_by_is_title_bout AS lose_pct_by_is_title_bout_ratio,
    t2.wins_by_weight_class - t3.wins_by_weight_class AS wins_by_weight_class_diff,
    1.0 * t2.wins_by_weight_class / t3.wins_by_weight_class AS wins_by_weight_class_ratio,
    t2.losses_by_weight_class - t3.losses_by_weight_class AS losses_by_weight_class_diff,
    1.0 * t2.losses_by_weight_class / t3.losses_by_weight_class AS losses_by_weight_class_ratio,
    t2.win_pct_by_weight_class - t3.win_pct_by_weight_class AS win_pct_by_weight_class_diff,
    1.0 * t2.win_pct_by_weight_class / t3.win_pct_by_weight_class AS win_pct_by_weight_class_ratio,
    t2.lose_pct_by_weight_class - t3.lose_pct_by_weight_class AS lose_pct_by_weight_class_diff,
    1.0 * t2.lose_pct_by_weight_class / t3.lose_pct_by_weight_class AS lose_pct_by_weight_class_ratio,
    t2.avg_opp_wins_by_country - t3.avg_opp_wins_by_country AS avg_opp_wins_by_country_diff,
    1.0 * t2.avg_opp_wins_by_country / t3.avg_opp_wins_by_country AS avg_opp_wins_by_country_ratio,
    t2.avg_opp_wins_by_country_diff - t3.avg_opp_wins_by_country_diff AS avg_opp_wins_by_country_diff_diff,
    1.0 * t2.avg_opp_wins_by_country_diff / t3.avg_opp_wins_by_country_diff AS avg_opp_wins_by_country_diff_ratio,
    t2.avg_opp_wins_by_country_ratio - t3.avg_opp_wins_by_country_ratio AS avg_opp_wins_by_country_ratio_diff,
    1.0 * t2.avg_opp_wins_by_country_ratio / t3.avg_opp_wins_by_country_ratio AS avg_opp_wins_by_country_ratio_ratio,
    t2.avg_opp_losses_by_country - t3.avg_opp_losses_by_country AS avg_opp_losses_by_country_diff,
    1.0 * t2.avg_opp_losses_by_country / t3.avg_opp_losses_by_country AS avg_opp_losses_by_country_ratio,
    t2.avg_opp_losses_by_country_diff - t3.avg_opp_losses_by_country_diff AS avg_opp_losses_by_country_diff_diff,
    1.0 * t2.avg_opp_losses_by_country_diff / t3.avg_opp_losses_by_country_diff AS avg_opp_losses_by_country_diff_ratio,
    t2.avg_opp_losses_by_country_ratio - t3.avg_opp_losses_by_country_ratio AS avg_opp_losses_by_country_ratio_diff,
    1.0 * t2.avg_opp_losses_by_country_ratio / t3.avg_opp_losses_by_country_ratio AS avg_opp_losses_by_country_ratio_ratio,
    t2.avg_opp_win_pct_by_country - t3.avg_opp_win_pct_by_country AS avg_opp_win_pct_by_country_diff,
    1.0 * t2.avg_opp_win_pct_by_country / t3.avg_opp_win_pct_by_country AS avg_opp_win_pct_by_country_ratio,
    t2.avg_opp_win_pct_by_country_diff - t3.avg_opp_win_pct_by_country_diff AS avg_opp_win_pct_by_country_diff_diff,
    1.0 * t2.avg_opp_win_pct_by_country_diff / t3.avg_opp_win_pct_by_country_diff AS avg_opp_win_pct_by_country_diff_ratio,
    t2.avg_opp_win_pct_by_country_ratio - t3.avg_opp_win_pct_by_country_ratio AS avg_opp_win_pct_by_country_ratio_diff,
    1.0 * t2.avg_opp_win_pct_by_country_ratio / t3.avg_opp_win_pct_by_country_ratio AS avg_opp_win_pct_by_country_ratio_ratio,
    t2.avg_opp_lose_pct_by_country - t3.avg_opp_lose_pct_by_country AS avg_opp_lose_pct_by_country_diff,
    1.0 * t2.avg_opp_lose_pct_by_country / t3.avg_opp_lose_pct_by_country AS avg_opp_lose_pct_by_country_ratio,
    t2.avg_opp_lose_pct_by_country_diff - t3.avg_opp_lose_pct_by_country_diff AS avg_opp_lose_pct_by_country_diff_diff,
    1.0 * t2.avg_opp_lose_pct_by_country_diff / t3.avg_opp_lose_pct_by_country_diff AS avg_opp_lose_pct_by_country_diff_ratio,
    t2.avg_opp_lose_pct_by_country_ratio - t3.avg_opp_lose_pct_by_country_ratio AS avg_opp_lose_pct_by_country_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_by_country_ratio / t3.avg_opp_lose_pct_by_country_ratio AS avg_opp_lose_pct_by_country_ratio_ratio,
    t2.avg_opp_wins_against_opp_nationality - t3.avg_opp_wins_against_opp_nationality AS avg_opp_wins_against_opp_nationality_diff,
    1.0 * t2.avg_opp_wins_against_opp_nationality / t3.avg_opp_wins_against_opp_nationality AS avg_opp_wins_against_opp_nationality_ratio,
    t2.avg_opp_wins_against_opp_nationality_diff - t3.avg_opp_wins_against_opp_nationality_diff AS avg_opp_wins_against_opp_nationality_diff_diff,
    1.0 * t2.avg_opp_wins_against_opp_nationality_diff / t3.avg_opp_wins_against_opp_nationality_diff AS avg_opp_wins_against_opp_nationality_diff_ratio,
    t2.avg_opp_wins_against_opp_nationality_ratio - t3.avg_opp_wins_against_opp_nationality_ratio AS avg_opp_wins_against_opp_nationality_ratio_diff,
    1.0 * t2.avg_opp_wins_against_opp_nationality_ratio / t3.avg_opp_wins_against_opp_nationality_ratio AS avg_opp_wins_against_opp_nationality_ratio_ratio,
    t2.avg_opp_losses_against_opp_nationality - t3.avg_opp_losses_against_opp_nationality AS avg_opp_losses_against_opp_nationality_diff,
    1.0 * t2.avg_opp_losses_against_opp_nationality / t3.avg_opp_losses_against_opp_nationality AS avg_opp_losses_against_opp_nationality_ratio,
    t2.avg_opp_losses_against_opp_nationality_diff - t3.avg_opp_losses_against_opp_nationality_diff AS avg_opp_losses_against_opp_nationality_diff_diff,
    1.0 * t2.avg_opp_losses_against_opp_nationality_diff / t3.avg_opp_losses_against_opp_nationality_diff AS avg_opp_losses_against_opp_nationality_diff_ratio,
    t2.avg_opp_losses_against_opp_nationality_ratio - t3.avg_opp_losses_against_opp_nationality_ratio AS avg_opp_losses_against_opp_nationality_ratio_diff,
    1.0 * t2.avg_opp_losses_against_opp_nationality_ratio / t3.avg_opp_losses_against_opp_nationality_ratio AS avg_opp_losses_against_opp_nationality_ratio_ratio,
    t2.avg_opp_win_pct_against_opp_nationality - t3.avg_opp_win_pct_against_opp_nationality AS avg_opp_win_pct_against_opp_nationality_diff,
    1.0 * t2.avg_opp_win_pct_against_opp_nationality / t3.avg_opp_win_pct_against_opp_nationality AS avg_opp_win_pct_against_opp_nationality_ratio,
    t2.avg_opp_win_pct_against_opp_nationality_diff - t3.avg_opp_win_pct_against_opp_nationality_diff AS avg_opp_win_pct_against_opp_nationality_diff_diff,
    1.0 * t2.avg_opp_win_pct_against_opp_nationality_diff / t3.avg_opp_win_pct_against_opp_nationality_diff AS avg_opp_win_pct_against_opp_nationality_diff_ratio,
    t2.avg_opp_win_pct_against_opp_nationality_ratio - t3.avg_opp_win_pct_against_opp_nationality_ratio AS avg_opp_win_pct_against_opp_nationality_ratio_diff,
    1.0 * t2.avg_opp_win_pct_against_opp_nationality_ratio / t3.avg_opp_win_pct_against_opp_nationality_ratio AS avg_opp_win_pct_against_opp_nationality_ratio_ratio,
    t2.avg_opp_lose_pct_against_opp_nationality - t3.avg_opp_lose_pct_against_opp_nationality AS avg_opp_lose_pct_against_opp_nationality_diff,
    1.0 * t2.avg_opp_lose_pct_against_opp_nationality / t3.avg_opp_lose_pct_against_opp_nationality AS avg_opp_lose_pct_against_opp_nationality_ratio,
    t2.avg_opp_lose_pct_against_opp_nationality_diff - t3.avg_opp_lose_pct_against_opp_nationality_diff AS avg_opp_lose_pct_against_opp_nationality_diff_diff,
    1.0 * t2.avg_opp_lose_pct_against_opp_nationality_diff / t3.avg_opp_lose_pct_against_opp_nationality_diff AS avg_opp_lose_pct_against_opp_nationality_diff_ratio,
    t2.avg_opp_lose_pct_against_opp_nationality_ratio - t3.avg_opp_lose_pct_against_opp_nationality_ratio AS avg_opp_lose_pct_against_opp_nationality_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_against_opp_nationality_ratio / t3.avg_opp_lose_pct_against_opp_nationality_ratio AS avg_opp_lose_pct_against_opp_nationality_ratio_ratio,
    t2.avg_opp_wins_by_is_home_country - t3.avg_opp_wins_by_is_home_country AS avg_opp_wins_by_is_home_country_diff,
    1.0 * t2.avg_opp_wins_by_is_home_country / t3.avg_opp_wins_by_is_home_country AS avg_opp_wins_by_is_home_country_ratio,
    t2.avg_opp_wins_by_is_home_country_diff - t3.avg_opp_wins_by_is_home_country_diff AS avg_opp_wins_by_is_home_country_diff_diff,
    1.0 * t2.avg_opp_wins_by_is_home_country_diff / t3.avg_opp_wins_by_is_home_country_diff AS avg_opp_wins_by_is_home_country_diff_ratio,
    t2.avg_opp_wins_by_is_home_country_ratio - t3.avg_opp_wins_by_is_home_country_ratio AS avg_opp_wins_by_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_wins_by_is_home_country_ratio / t3.avg_opp_wins_by_is_home_country_ratio AS avg_opp_wins_by_is_home_country_ratio_ratio,
    t2.avg_opp_losses_by_is_home_country - t3.avg_opp_losses_by_is_home_country AS avg_opp_losses_by_is_home_country_diff,
    1.0 * t2.avg_opp_losses_by_is_home_country / t3.avg_opp_losses_by_is_home_country AS avg_opp_losses_by_is_home_country_ratio,
    t2.avg_opp_losses_by_is_home_country_diff - t3.avg_opp_losses_by_is_home_country_diff AS avg_opp_losses_by_is_home_country_diff_diff,
    1.0 * t2.avg_opp_losses_by_is_home_country_diff / t3.avg_opp_losses_by_is_home_country_diff AS avg_opp_losses_by_is_home_country_diff_ratio,
    t2.avg_opp_losses_by_is_home_country_ratio - t3.avg_opp_losses_by_is_home_country_ratio AS avg_opp_losses_by_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_losses_by_is_home_country_ratio / t3.avg_opp_losses_by_is_home_country_ratio AS avg_opp_losses_by_is_home_country_ratio_ratio,
    t2.avg_opp_win_pct_by_is_home_country - t3.avg_opp_win_pct_by_is_home_country AS avg_opp_win_pct_by_is_home_country_diff,
    1.0 * t2.avg_opp_win_pct_by_is_home_country / t3.avg_opp_win_pct_by_is_home_country AS avg_opp_win_pct_by_is_home_country_ratio,
    t2.avg_opp_win_pct_by_is_home_country_diff - t3.avg_opp_win_pct_by_is_home_country_diff AS avg_opp_win_pct_by_is_home_country_diff_diff,
    1.0 * t2.avg_opp_win_pct_by_is_home_country_diff / t3.avg_opp_win_pct_by_is_home_country_diff AS avg_opp_win_pct_by_is_home_country_diff_ratio,
    t2.avg_opp_win_pct_by_is_home_country_ratio - t3.avg_opp_win_pct_by_is_home_country_ratio AS avg_opp_win_pct_by_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_win_pct_by_is_home_country_ratio / t3.avg_opp_win_pct_by_is_home_country_ratio AS avg_opp_win_pct_by_is_home_country_ratio_ratio,
    t2.avg_opp_lose_pct_by_is_home_country - t3.avg_opp_lose_pct_by_is_home_country AS avg_opp_lose_pct_by_is_home_country_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_home_country / t3.avg_opp_lose_pct_by_is_home_country AS avg_opp_lose_pct_by_is_home_country_ratio,
    t2.avg_opp_lose_pct_by_is_home_country_diff - t3.avg_opp_lose_pct_by_is_home_country_diff AS avg_opp_lose_pct_by_is_home_country_diff_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_home_country_diff / t3.avg_opp_lose_pct_by_is_home_country_diff AS avg_opp_lose_pct_by_is_home_country_diff_ratio,
    t2.avg_opp_lose_pct_by_is_home_country_ratio - t3.avg_opp_lose_pct_by_is_home_country_ratio AS avg_opp_lose_pct_by_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_home_country_ratio / t3.avg_opp_lose_pct_by_is_home_country_ratio AS avg_opp_lose_pct_by_is_home_country_ratio_ratio,
    t2.avg_opp_wins_by_opp_is_home_country - t3.avg_opp_wins_by_opp_is_home_country AS avg_opp_wins_by_opp_is_home_country_diff,
    1.0 * t2.avg_opp_wins_by_opp_is_home_country / t3.avg_opp_wins_by_opp_is_home_country AS avg_opp_wins_by_opp_is_home_country_ratio,
    t2.avg_opp_wins_by_opp_is_home_country_diff - t3.avg_opp_wins_by_opp_is_home_country_diff AS avg_opp_wins_by_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_wins_by_opp_is_home_country_diff / t3.avg_opp_wins_by_opp_is_home_country_diff AS avg_opp_wins_by_opp_is_home_country_diff_ratio,
    t2.avg_opp_wins_by_opp_is_home_country_ratio - t3.avg_opp_wins_by_opp_is_home_country_ratio AS avg_opp_wins_by_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_wins_by_opp_is_home_country_ratio / t3.avg_opp_wins_by_opp_is_home_country_ratio AS avg_opp_wins_by_opp_is_home_country_ratio_ratio,
    t2.avg_opp_losses_by_opp_is_home_country - t3.avg_opp_losses_by_opp_is_home_country AS avg_opp_losses_by_opp_is_home_country_diff,
    1.0 * t2.avg_opp_losses_by_opp_is_home_country / t3.avg_opp_losses_by_opp_is_home_country AS avg_opp_losses_by_opp_is_home_country_ratio,
    t2.avg_opp_losses_by_opp_is_home_country_diff - t3.avg_opp_losses_by_opp_is_home_country_diff AS avg_opp_losses_by_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_losses_by_opp_is_home_country_diff / t3.avg_opp_losses_by_opp_is_home_country_diff AS avg_opp_losses_by_opp_is_home_country_diff_ratio,
    t2.avg_opp_losses_by_opp_is_home_country_ratio - t3.avg_opp_losses_by_opp_is_home_country_ratio AS avg_opp_losses_by_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_losses_by_opp_is_home_country_ratio / t3.avg_opp_losses_by_opp_is_home_country_ratio AS avg_opp_losses_by_opp_is_home_country_ratio_ratio,
    t2.avg_opp_win_pct_by_opp_is_home_country - t3.avg_opp_win_pct_by_opp_is_home_country AS avg_opp_win_pct_by_opp_is_home_country_diff,
    1.0 * t2.avg_opp_win_pct_by_opp_is_home_country / t3.avg_opp_win_pct_by_opp_is_home_country AS avg_opp_win_pct_by_opp_is_home_country_ratio,
    t2.avg_opp_win_pct_by_opp_is_home_country_diff - t3.avg_opp_win_pct_by_opp_is_home_country_diff AS avg_opp_win_pct_by_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_win_pct_by_opp_is_home_country_diff / t3.avg_opp_win_pct_by_opp_is_home_country_diff AS avg_opp_win_pct_by_opp_is_home_country_diff_ratio,
    t2.avg_opp_win_pct_by_opp_is_home_country_ratio - t3.avg_opp_win_pct_by_opp_is_home_country_ratio AS avg_opp_win_pct_by_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_win_pct_by_opp_is_home_country_ratio / t3.avg_opp_win_pct_by_opp_is_home_country_ratio AS avg_opp_win_pct_by_opp_is_home_country_ratio_ratio,
    t2.avg_opp_lose_pct_by_opp_is_home_country - t3.avg_opp_lose_pct_by_opp_is_home_country AS avg_opp_lose_pct_by_opp_is_home_country_diff,
    1.0 * t2.avg_opp_lose_pct_by_opp_is_home_country / t3.avg_opp_lose_pct_by_opp_is_home_country AS avg_opp_lose_pct_by_opp_is_home_country_ratio,
    t2.avg_opp_lose_pct_by_opp_is_home_country_diff - t3.avg_opp_lose_pct_by_opp_is_home_country_diff AS avg_opp_lose_pct_by_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_lose_pct_by_opp_is_home_country_diff / t3.avg_opp_lose_pct_by_opp_is_home_country_diff AS avg_opp_lose_pct_by_opp_is_home_country_diff_ratio,
    t2.avg_opp_lose_pct_by_opp_is_home_country_ratio - t3.avg_opp_lose_pct_by_opp_is_home_country_ratio AS avg_opp_lose_pct_by_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_by_opp_is_home_country_ratio / t3.avg_opp_lose_pct_by_opp_is_home_country_ratio AS avg_opp_lose_pct_by_opp_is_home_country_ratio_ratio,
    t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country - t3.avg_opp_wins_by_is_home_country_and_opp_is_home_country AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country / t3.avg_opp_wins_by_is_home_country_and_opp_is_home_country AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio,
    t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff - t3.avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff / t3.avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_diff_ratio,
    t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio - t3.avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio / t3.avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_wins_by_is_home_country_and_opp_is_home_country_ratio_ratio,
    t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country - t3.avg_opp_losses_by_is_home_country_and_opp_is_home_country AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country / t3.avg_opp_losses_by_is_home_country_and_opp_is_home_country AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio,
    t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff - t3.avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff / t3.avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_diff_ratio,
    t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio - t3.avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio / t3.avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_losses_by_is_home_country_and_opp_is_home_country_ratio_ratio,
    t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country - t3.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country / t3.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio,
    t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff - t3.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff / t3.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_diff_ratio,
    t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio - t3.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio / t3.avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_win_pct_by_is_home_country_and_opp_is_home_country_ratio_ratio,
    t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country - t3.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country / t3.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio,
    t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff - t3.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff / t3.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_diff_ratio,
    t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio - t3.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio / t3.avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio AS avg_opp_lose_pct_by_is_home_country_and_opp_is_home_country_ratio_ratio,
    t2.avg_opp_wins_by_is_title_bout - t3.avg_opp_wins_by_is_title_bout AS avg_opp_wins_by_is_title_bout_diff,
    1.0 * t2.avg_opp_wins_by_is_title_bout / t3.avg_opp_wins_by_is_title_bout AS avg_opp_wins_by_is_title_bout_ratio,
    t2.avg_opp_wins_by_is_title_bout_diff - t3.avg_opp_wins_by_is_title_bout_diff AS avg_opp_wins_by_is_title_bout_diff_diff,
    1.0 * t2.avg_opp_wins_by_is_title_bout_diff / t3.avg_opp_wins_by_is_title_bout_diff AS avg_opp_wins_by_is_title_bout_diff_ratio,
    t2.avg_opp_wins_by_is_title_bout_ratio - t3.avg_opp_wins_by_is_title_bout_ratio AS avg_opp_wins_by_is_title_bout_ratio_diff,
    1.0 * t2.avg_opp_wins_by_is_title_bout_ratio / t3.avg_opp_wins_by_is_title_bout_ratio AS avg_opp_wins_by_is_title_bout_ratio_ratio,
    t2.avg_opp_losses_by_is_title_bout - t3.avg_opp_losses_by_is_title_bout AS avg_opp_losses_by_is_title_bout_diff,
    1.0 * t2.avg_opp_losses_by_is_title_bout / t3.avg_opp_losses_by_is_title_bout AS avg_opp_losses_by_is_title_bout_ratio,
    t2.avg_opp_losses_by_is_title_bout_diff - t3.avg_opp_losses_by_is_title_bout_diff AS avg_opp_losses_by_is_title_bout_diff_diff,
    1.0 * t2.avg_opp_losses_by_is_title_bout_diff / t3.avg_opp_losses_by_is_title_bout_diff AS avg_opp_losses_by_is_title_bout_diff_ratio,
    t2.avg_opp_losses_by_is_title_bout_ratio - t3.avg_opp_losses_by_is_title_bout_ratio AS avg_opp_losses_by_is_title_bout_ratio_diff,
    1.0 * t2.avg_opp_losses_by_is_title_bout_ratio / t3.avg_opp_losses_by_is_title_bout_ratio AS avg_opp_losses_by_is_title_bout_ratio_ratio,
    t2.avg_opp_win_pct_by_is_title_bout - t3.avg_opp_win_pct_by_is_title_bout AS avg_opp_win_pct_by_is_title_bout_diff,
    1.0 * t2.avg_opp_win_pct_by_is_title_bout / t3.avg_opp_win_pct_by_is_title_bout AS avg_opp_win_pct_by_is_title_bout_ratio,
    t2.avg_opp_win_pct_by_is_title_bout_diff - t3.avg_opp_win_pct_by_is_title_bout_diff AS avg_opp_win_pct_by_is_title_bout_diff_diff,
    1.0 * t2.avg_opp_win_pct_by_is_title_bout_diff / t3.avg_opp_win_pct_by_is_title_bout_diff AS avg_opp_win_pct_by_is_title_bout_diff_ratio,
    t2.avg_opp_win_pct_by_is_title_bout_ratio - t3.avg_opp_win_pct_by_is_title_bout_ratio AS avg_opp_win_pct_by_is_title_bout_ratio_diff,
    1.0 * t2.avg_opp_win_pct_by_is_title_bout_ratio / t3.avg_opp_win_pct_by_is_title_bout_ratio AS avg_opp_win_pct_by_is_title_bout_ratio_ratio,
    t2.avg_opp_lose_pct_by_is_title_bout - t3.avg_opp_lose_pct_by_is_title_bout AS avg_opp_lose_pct_by_is_title_bout_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_title_bout / t3.avg_opp_lose_pct_by_is_title_bout AS avg_opp_lose_pct_by_is_title_bout_ratio,
    t2.avg_opp_lose_pct_by_is_title_bout_diff - t3.avg_opp_lose_pct_by_is_title_bout_diff AS avg_opp_lose_pct_by_is_title_bout_diff_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_title_bout_diff / t3.avg_opp_lose_pct_by_is_title_bout_diff AS avg_opp_lose_pct_by_is_title_bout_diff_ratio,
    t2.avg_opp_lose_pct_by_is_title_bout_ratio - t3.avg_opp_lose_pct_by_is_title_bout_ratio AS avg_opp_lose_pct_by_is_title_bout_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_by_is_title_bout_ratio / t3.avg_opp_lose_pct_by_is_title_bout_ratio AS avg_opp_lose_pct_by_is_title_bout_ratio_ratio,
    t2.avg_opp_wins_by_weight_class - t3.avg_opp_wins_by_weight_class AS avg_opp_wins_by_weight_class_diff,
    1.0 * t2.avg_opp_wins_by_weight_class / t3.avg_opp_wins_by_weight_class AS avg_opp_wins_by_weight_class_ratio,
    t2.avg_opp_wins_by_weight_class_diff - t3.avg_opp_wins_by_weight_class_diff AS avg_opp_wins_by_weight_class_diff_diff,
    1.0 * t2.avg_opp_wins_by_weight_class_diff / t3.avg_opp_wins_by_weight_class_diff AS avg_opp_wins_by_weight_class_diff_ratio,
    t2.avg_opp_wins_by_weight_class_ratio - t3.avg_opp_wins_by_weight_class_ratio AS avg_opp_wins_by_weight_class_ratio_diff,
    1.0 * t2.avg_opp_wins_by_weight_class_ratio / t3.avg_opp_wins_by_weight_class_ratio AS avg_opp_wins_by_weight_class_ratio_ratio,
    t2.avg_opp_losses_by_weight_class - t3.avg_opp_losses_by_weight_class AS avg_opp_losses_by_weight_class_diff,
    1.0 * t2.avg_opp_losses_by_weight_class / t3.avg_opp_losses_by_weight_class AS avg_opp_losses_by_weight_class_ratio,
    t2.avg_opp_losses_by_weight_class_diff - t3.avg_opp_losses_by_weight_class_diff AS avg_opp_losses_by_weight_class_diff_diff,
    1.0 * t2.avg_opp_losses_by_weight_class_diff / t3.avg_opp_losses_by_weight_class_diff AS avg_opp_losses_by_weight_class_diff_ratio,
    t2.avg_opp_losses_by_weight_class_ratio - t3.avg_opp_losses_by_weight_class_ratio AS avg_opp_losses_by_weight_class_ratio_diff,
    1.0 * t2.avg_opp_losses_by_weight_class_ratio / t3.avg_opp_losses_by_weight_class_ratio AS avg_opp_losses_by_weight_class_ratio_ratio,
    t2.avg_opp_win_pct_by_weight_class - t3.avg_opp_win_pct_by_weight_class AS avg_opp_win_pct_by_weight_class_diff,
    1.0 * t2.avg_opp_win_pct_by_weight_class / t3.avg_opp_win_pct_by_weight_class AS avg_opp_win_pct_by_weight_class_ratio,
    t2.avg_opp_win_pct_by_weight_class_diff - t3.avg_opp_win_pct_by_weight_class_diff AS avg_opp_win_pct_by_weight_class_diff_diff,
    1.0 * t2.avg_opp_win_pct_by_weight_class_diff / t3.avg_opp_win_pct_by_weight_class_diff AS avg_opp_win_pct_by_weight_class_diff_ratio,
    t2.avg_opp_win_pct_by_weight_class_ratio - t3.avg_opp_win_pct_by_weight_class_ratio AS avg_opp_win_pct_by_weight_class_ratio_diff,
    1.0 * t2.avg_opp_win_pct_by_weight_class_ratio / t3.avg_opp_win_pct_by_weight_class_ratio AS avg_opp_win_pct_by_weight_class_ratio_ratio,
    t2.avg_opp_lose_pct_by_weight_class - t3.avg_opp_lose_pct_by_weight_class AS avg_opp_lose_pct_by_weight_class_diff,
    1.0 * t2.avg_opp_lose_pct_by_weight_class / t3.avg_opp_lose_pct_by_weight_class AS avg_opp_lose_pct_by_weight_class_ratio,
    t2.avg_opp_lose_pct_by_weight_class_diff - t3.avg_opp_lose_pct_by_weight_class_diff AS avg_opp_lose_pct_by_weight_class_diff_diff,
    1.0 * t2.avg_opp_lose_pct_by_weight_class_diff / t3.avg_opp_lose_pct_by_weight_class_diff AS avg_opp_lose_pct_by_weight_class_diff_ratio,
    t2.avg_opp_lose_pct_by_weight_class_ratio - t3.avg_opp_lose_pct_by_weight_class_ratio AS avg_opp_lose_pct_by_weight_class_ratio_diff,
    1.0 * t2.avg_opp_lose_pct_by_weight_class_ratio / t3.avg_opp_lose_pct_by_weight_class_ratio AS avg_opp_lose_pct_by_weight_class_ratio_ratio,
    CASE
        WHEN red_outcome = 'W' THEN 1
        ELSE 0
    END AS red_win
FROM ufcstats_bouts AS t1
LEFT JOIN cte12 AS t2 ON t1.id = t2.bout_id AND t1.red_fighter_id = t2.fighter_id
LEFT JOIN cte12 AS t3 ON t1.id = t3.bout_id AND t1.blue_fighter_id = t3.fighter_id
WHERE event_id IN (
    SELECT id FROM ufcstats_events
    WHERE is_ufc_event = 1 AND date >= '2008-04-19' AND date < '2021-01-01'
) AND red_outcome IN ('W', 'L') AND outcome_method != 'DQ'
"""

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql(query, conn)
df

,id,wins_by_country_diff,wins_by_country_ratio,losses_by_country_diff,losses_by_country_ratio,win_pct_by_country_diff,win_pct_by_country_ratio,lose_pct_by_country_diff,lose_pct_by_country_ratio,wins_against_opp_nationality_diff,...,avg_opp_win_pct_by_weight_class_diff_ratio,avg_opp_win_pct_by_weight_class_ratio_diff,avg_opp_win_pct_by_weight_class_ratio_ratio,avg_opp_lose_pct_by_weight_class_diff,avg_opp_lose_pct_by_weight_class_ratio,avg_opp_lose_pct_by_weight_class_diff_diff,avg_opp_lose_pct_by_weight_class_diff_ratio,avg_opp_lose_pct_by_weight_class_ratio_diff,avg_opp_lose_pct_by_weight_class_ratio_ratio,red_win
0,be38ed9ccfe2ee03,17,NaN,4,NaN,0.039394,1.053719,-0.084848,0.681818,0,...,-0.349691,-0.280289,0.785499,-0.036487,0.873127,0.203404,-1.032884,0.531523,1.697601,1
1,eb1b371dfc37fcdb,-1,0.000000,0,NaN,0.000000,1.000000,0.000000,NaN,0,...,0.000000,-0.444805,0.692135,-0.239881,0.409956,0.152769,-0.000000,NaN,NaN,1
2,219bd976b8ca745d,10,11.000000,0,NaN,0.000000,1.000000,0.000000,NaN,3,...,5.259168,-0.282523,0.778997,-0.057416,0.833528,-0.105081,-1.673595,-0.385064,0.713595,0
3,af178adff964d854,0,NaN,0,NaN,-0.238095,0.761905,0.238095,NaN,-1,...,0.620744,-0.148414,0.890502,0.135687,1.670265,0.094868,0.531373,0.727443,NaN,0
4,920194911d727a38,-17,0.000000,-7,0.0,0.041667,1.058824,-0.041667,0.857143,-1,...,-2.343761,0.276584,1.291260,0.038213,1.117190,-0.144433,-1.536244,-0.364229,0.727350,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4907,cd150cd28738a7c5,-4,0.428571,0,1.0,-0.027778,0.964286,0.138889,2.250000,1,...,2.878955,0.274779,1.243350,0.047414,1.242468,-0.060378,1.750836,0.126492,1.204165,1
4908,8955ea3c7c332e6c,0,1.000000,1,1.5,-0.044737,0.950000,0.044737,1.425000,10,...,0.043623,-0.127167,0.886207,-0.093887,0.663360,0.071606,0.294599,-0.375144,0.694889,0
4909,3d35eb2d46bf74de,-8,0.200000,0,1.0,-0.242424,0.733333,0.242424,3.666667,0,...,0.281492,-0.860903,0.577848,-0.184009,0.666012,0.272309,0.297059,0.516618,2.787883,1
4910,014f1da2083ca174,5,1.714286,2,3.0,-0.075000,0.914286,0.075000,1.600000,0,...,0.791788,-0.075547,0.940405,-0.149224,0.538848,0.040687,0.675511,0.259774,1.373293,1
